In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Import the ModelCheckpoint callback
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:

# Define class names
class_names = ['Automatic Rifle', 'Bows or Arrows', 'Knifes', 'Short Gun', 'Sniper', 'Spears or Polearms', 'SubMachine Gun', 'Sword', 'pistols']

# Load pre-trained VGG19 model (without top layers)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers (optional, but often improves stability)
for layer in base_model.layers:
    layer.trainable = False

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:

# Add custom layers for multi-label classification
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)  # Adjust units as needed
x = Dropout(0.5)(x)
predictions = Dense(len(class_names), activation='sigmoid')(x)  # Sigmoid for multi-label classification

model_sample = Model(inputs=base_model.input, outputs=predictions)

model_sample.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',  # Use binary crossentropy for multi-label classification
              metrics=['accuracy'])


In [ ]:


def load_and_preprocess_data(data_dir):

  # Create an ImageDataGenerator for preprocessing
  datagen = ImageDataGenerator(
      rescale=1./255,
  )

  # Load and preprocess the data
  data_generator = datagen.flow_from_directory(
      data_dir,
      target_size=(224, 224),
      batch_size=32,
      class_mode='categorical'  # Use 'categorical' for multi-label classification
  )

  return data_generator

In [ ]:




train_data_path = '/content/drive/MyDrive/Resume Projects OpenCV/Weapons Classification using VGG19 (Pre-trained)/train_S'
val_data_path = '/content/drive/MyDrive/Resume Projects OpenCV/Weapons Classification using VGG19 (Pre-trained)/test_S'

# Use the correct path variables when calling load_and_preprocess_data
train_generator = load_and_preprocess_data(train_data_path)
val_generator = load_and_preprocess_data(val_data_path)

checkpoint_path = "best_weights_sample.weights.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',  # Monitor validation accuracy
    save_best_only=True,  # Save only the model with the best validation accuracy
    save_weights_only=True,  # Only save model weights, not the entire model
    mode='max'  # Maximize validation accuracy
)

# Train the model
epochs = 30  # Adjust epochs
history = model_sample.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[checkpoint_callback]
)

# Save the final weights
model_sample.save_weights('final_weights_sample.weights.h5')

# Save the model (optional)
model_sample.save('vgg19_multilabel_model_sample.h5')


Found 1567 images belonging to 9 classes.
Found 882 images belonging to 9 classes.
Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 7/49 ━━━━━━━━━━━━━━━━━━━━ 16:35 24s/step - accuracy: 0.1349 - loss: 0.6209